# Python

![](https://leukipp.github.io/xmastree/kernelspecs/python.png)

This file is just for reference purpose.
Code here doesn`t run as some required packages and system libraries are missing!

In [ ]:
import os
import csv
import cairosvg
import numpy as np
import matplotlib.pyplot as plt

from drawSvg.animation import AnimationContext
from drawSvg import Drawing, Rectangle, Lines, Text, LinearGradient


## Utils

In [ ]:

class Drawing(Drawing):
    def __init__(self, width, height, **svgArgs):
        """ extend drawing class for GIFT support """
        super().__init__(width, height, **svgArgs)

    def asNumpy(self, alpha=False, dpi=96):
        """ get RGB(A) numpy image array from svg """
        tree = cairosvg.parser.Tree(bytestring=self.asSvg())
        cairo = cairosvg.surface.PNGSurface(tree, None, dpi).cairo
        shape = (cairo.get_height(), cairo.get_width(), 4)

        # convert color channels
        image_bgra = +np.frombuffer(cairo.get_data(), np.uint8)
        image_rgba = image_bgra.reshape(shape)[:, :, [2, 1, 0, 3]]

        return image_rgba[:, :, :3] if not alpha else image_rgba

    def colorCoords(self, coords, plane=(1, 2)):
        """ map RGB(A) colors to given coords """
        img = self.asNumpy()

        # use defined 2D plane with y negated
        x, y = coords[:, plane[0]], -coords[:, plane[1]]

        # image w-h is index j-i
        idxs_j = np.interp(x, [x.min(), x.max()], [0, img.shape[1] - 1]).round().astype(int)
        idxs_i = np.interp(y, [y.min(), y.max()], [0, img.shape[0] - 1]).round().astype(int)

        return img[idxs_i, idxs_j]

    def plotCoords(self, coords):
        """ plot RGB(A) colored coords """
        colors = self.colorCoords(coords)
        x, y, z = np.split(coords, 3, axis=1)

        # mask unused coordinates
        mask = ~np.all(colors == [0, 0, 0], axis=1)
        coords_masked = coords[mask]
        colors_masked = colors[mask]

        # plot coordinates
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(projection='3d')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        ax.set_xlim([x.min(), x.max()])
        ax.set_ylim([y.min(), y.max()])
        ax.set_zlim([z.min(), z.max()])
        ax.view_init(elev=0, azim=0)
        ax.scatter(*np.split(coords_masked, 3, axis=1), color=colors_masked / 255)


class AnimationContext(AnimationContext):
    def __init__(self, coords, draw_func, **kwargs):
        """ extend animation context class for GIFT support """
        super().__init__(draw_func=draw_func, **kwargs)
        self.coords = coords
        self.frames = []

    def __exit__(self, exc_type, exc_value, exc_traceback):
        """ save RGB(A) colored frames """
        super().__exit__(exc_type, exc_value, exc_traceback)
        self.frames = [d.colorCoords(self.coords) for d in self.anim.frames]

    def export(self, path):
        """ export RGB(A) colored coords as csv """
        head = ['FRAME_ID'] + np.hstack([[f'R_{i}', f'G_{i}', f'B_{i}'] for i in range(self.coords.shape[0])]).tolist()
        rows = np.array(self.frames, dtype=int)

        # build csv data
        ids = np.arange(rows.shape[0]).reshape(-1, 1)
        frames = np.reshape(rows, (*rows.shape[:1], -1))
        data = np.hstack([ids, frames])

        # export csv
        np.savetxt(path, data, header=','.join(head), fmt='%i', delimiter=',', comments='')


## Setup

In [ ]:
# init coords
with open('coords_2021.csv', 'r', encoding='utf-8-sig') as f:
    coords_csv = list(csv.reader(f))
coords = np.array([[float(col) for col in row] for row in coords_csv])


## Code

### Rocket launch

[github.com/cduck/drawSvg](https://github.com/cduck/drawSvg)

Tests of using a python library to programmatically generate SVG images and render them as GIFT animation frames.

In [ ]:
# ------------------------------------------ START CODING ------------------------------------------

fps = 20

def draw_func(t=0):
    """ main drawing function """
    d = Drawing(100, 150)

    counter = np.interp(t, [1, 10 * fps], [1, 10]).astype(int)
    if counter < 10:
        # countdown text
        d.append(Text(str(10 - counter), 90, 20, 30, fill='#00ff00'))
    else:
        t -= 10 * fps
        delay = 2 * fps

        # calculate deltas
        dy = 1 * (t - delay if t >= delay else 0)
        dx = np.minimum(30, t)

        # engine colors
        g = LinearGradient(d.width / 2, 30 + dy, d.width / 2, -10)
        g.addStop(0, '#ff7700', 1)
        g.addStop(1, '#ff0000', 0)

        # engine polygon
        d.append(Lines(
            d.width / 2, 100 + dy,
            d.width / 2 - dx, dy - dx,
            d.width / 2 + dx, dy - dx,
            close=True, fill=g))

        # rocket rectangle
        d.append(Rectangle(0, 15 + dy, 100, 135, fill='#0055aa'))

    return d

# animate drawing function and export csv
ctx = AnimationContext(coords, draw_func, pause=False, clear=True, delay=0.05, jupyter=True, out_file='rocket_launch.gif')
with ctx as animation:
    for t in np.arange(22 * fps):
        animation.draw_frame(t)
ctx.export('rocket_launch.csv')

# ------------------------------------------- END CODING -------------------------------------------
